# Lab 01: System Inference Profiles

## Business Context

You are **Alex, Platform Engineer at AnyCompany Solutions**, a SaaS company building a **Multi-Tenant Marketing Platform** that serves **multiple enterprise customers**. Your platform uses Claude Sonnet 4.0 to generate intelligent responses for various customer workloads.

**Current Setup:**
- Using AWS Bedrock with **System Inference Profiles**
- Multiple enterprise customers sharing the same model endpoint
- All customers route through `us.anthropic.claude-sonnet-4-20250514-v1:0`

**The Problem You're Facing:**

Your CEO just asked three critical questions:

1. **💰 "How much does each customer cost us in AI spending?"**
   - You can't answer - all usage is aggregated together
   
2. **📊 "Which customers are heavy users vs light users?"**
   - You don't know - no per-customer visibility
   
3. **🎯 "Can we implement usage-based pricing?"**
   - Not reliably - you can't track individual customer usage

**The Root Cause: System Inference Profiles**

When you use System Inference Profiles, AWS Bedrock treats all requests the same:
- ❌ No tenant identification in metrics
- ❌ No way to separate customer A's usage from customer B's
- ❌ CloudWatch metrics show only aggregated totals
- ❌ Impossible to implement accurate billing per customer

## Learning Objectives

By the end of this lab, you'll understand:
- How System Inference Profiles work (and their limitations)
- Why multi-tenant applications can't track per-customer metrics with System Inference Profiles
- What CloudWatch metrics look like with System Inference Profiles (aggregated data)
- The business impact of not having tenant-level visibility

## What You'll Do

1. **🔧 Make a model inference call** using System Inference Profile
2. **📊 Check CloudWatch metrics** and see aggregated (not separated) data
3. **❌ Experience the problem firsthand** - no way to attribute usage to specific customers
4. **💡 Understand why** this is a showstopper for multi-tenant SaaS platforms

**Ready to see the problem?** Let's start by making an inference call using System Inference Profiles...

## Section 1: Setup and Configuration

First, let's import the necessary libraries and configure our AWS Bedrock client with a **System Inference Profile** (the traditional approach).

In [ ]:
# Install required packages
!pip install --force-reinstall -r requirements.txt --quiet

In [ ]:
import boto3
import datetime, time
import sys
from datetime import timedelta
import matplotlib.pyplot as plt
from lab_helpers.config import Region, ModelId, Prompt

### 📋 Configuration Check

Let's verify our configuration from the lab helpers:
- **Region**: AWS region where Bedrock is available
- **ModelId**: System Inference Profile ID (shared across all customers)
- **Prompt**: Sample prompt to test the model

In [ ]:
print(f"AWS Region is {Region}")
print(f"Model ID is {ModelId}")

## Section 2: Make Inference Call with System Inference 

  ### 🤖 The Problem in Action

  Let's simulate what happens when multiple tenants use your platform with a **System Inference Profile**:

  - **Tenant A (B2B)**: Requests campaign for DevOps platform
  - **Tenant B (B2C)**: Requests campaign for fashion collection

  **❌ The Critical Issue:**
  Both tenants call the **same** `modelId` - there's no way to distinguish their usage in CloudWatch!

  ```python
  # Both tenants use this:
  modelId = "us.anthropic.claude-sonnet-4-20250514-v1:0"

  Result: All metrics are aggregated together.
  ```


In [ ]:
# Simulate multi-tenant requests using System Inference Profile
# In a real multi-tenant app, these would come from different customers

# Define prompts for two tenants (same as Lab 03 context)
TENANT_PROMPTS = {
    "tenant_a": "Generate a brief marketing campaign for a B2B SaaS DevOps automation platform targeting CTOs.",
    "tenant_b": "Generate a brief marketing campaign for a B2C sustainable fashion collection targeting millennials."
}

# Create Bedrock runtime client
bedrock = boto3.client('bedrock-runtime', region_name=Region)

print("🔄 Simulating requests from multiple tenants using System Inference Profile...")
print("=" * 80)

# Make inference calls for both tenants
for idx, (tenant_id, prompt) in enumerate(TENANT_PROMPTS.items()):
    print(f"\n🏢 Request from {tenant_id.upper()}:")
    print(f"Prompt: {prompt[:80]}...")
    
    # THE PROBLEM: Both tenants use the SAME modelId!
    response = bedrock.converse(
        modelId=ModelId,  # ❌ Same for all tenants - no way to distinguish!
        messages=[
            {
                'role': 'user',
                'content': [{'text': prompt}]
            }
        ]
    )
    
    # Extract response
    output = response['output']['message']['content'][0]['text']
    usage = response['usage']
    
    print(f"✅ Response received")
    print(f"   Input tokens: {usage['inputTokens']}")
    print(f"   Output tokens: {usage['outputTokens']}")
    print(f"   Response preview: {output[:100]}...")

    # Add 1-minute gap between tenant requests for CloudWatch visualization
    if idx == 0:  # After first tenant only
          print(f"\n⏳ Waiting 80 seconds before next request (for CloudWatch separation)...")
          time.sleep(60)
          print(f"✅ Wait complete - proceeding with Tenant B request")

print("\n" + "=" * 80)
print("THE PROBLEM: Both tenant requests used the SAME modelId!")
print(f"   ModelId: {ModelId}")
print("   All usage will be aggregated in CloudWatch - no way to separate!")
print("=" * 80)

## Section 3: Check CloudWatch Metrics - The Aggregation Problem

### 📊 CloudWatch Metrics Helper Function

This function fetches CloudWatch metrics from AWS Bedrock for the System Inference Profile.

**What it retrieves:**
- **Invocations**: Total number of API calls (all customers combined)
- **InputTokenCount**: Total input tokens processed (all customers combined)
- **OutputTokenCount**: Total output tokens generated (all customers combined)

**The Critical Limitation:**
All metrics use the **same ModelId dimension** - the System Inference Profile ID. This means:
- ❌ No way to filter by customer/tenant
- ❌ No way to see which customer made which request
- ❌ No way to allocate costs per customer

**Dimension in CloudWatch:** `ModelId = us.anthropic.claude-sonnet-4-20250514-v1:0`

This dimension is **shared by all customers**, making per-customer tracking impossible!

In [ ]:
# Create CloudWatch client

def fetch_metrices(Region, ModelId, Period=300, Timedelta=60):
    
    cloudwatch = boto3.client('cloudwatch', region_name=Region)
    
    # Get metrics for the last hour
    #end_time = datetime.utcnow()
    end_time=datetime.datetime.now(datetime.UTC)
    start_time = end_time - timedelta(minutes=60)
    
    # Get Bedrock invocation metrics
    #Invocations - Number of API calls
    response = cloudwatch.get_metric_statistics(
        Namespace='AWS/Bedrock',
        MetricName='Invocations',
        Dimensions=[
            {
                'Name': 'ModelId',
                'Value': ModelId
            }
        ],
        StartTime=start_time,
        EndTime=end_time,
        Period=60,  # 1 minute
        Statistics=['Sum']
    )
    
    print("Invocation Count:")
    for datapoint in response['Datapoints']:
        print(f"Time: {datapoint['Timestamp']}, Count: {datapoint['Sum']}")
    
    # Get input token metrics 
    # InputTokenCount - Total input tokens processed
    
    input_token_response = cloudwatch.get_metric_statistics(
        Namespace='AWS/Bedrock',
        MetricName='InputTokenCount',
        Dimensions=[
            {
                'Name': 'ModelId',
                'Value': ModelId
            }
        ],
        StartTime=start_time,
        EndTime=end_time,
        Period=60,
        Statistics=['Sum']
    )
    
    print("\nInput Token Count:")
    for datapoint in input_token_response['Datapoints']:
        print(f"Time: {datapoint['Timestamp']}, Tokens: {datapoint['Sum']}")
    
    # Get output token metrics
    #OutputTokenCount - Total output tokens generated
    output_token_response = cloudwatch.get_metric_statistics(
        Namespace='AWS/Bedrock',
        MetricName='OutputTokenCount',
        Dimensions=[
            {
                'Name': 'ModelId',
                'Value': ModelId
            }
        ],
        StartTime=start_time,
        EndTime=end_time,
        Period=60,
        Statistics=['Sum']
    )
    
    print("\nOutput Token Count:")
    for datapoint in output_token_response['Datapoints']:
        print(f"Time: {datapoint['Timestamp']}, Tokens: {datapoint['Sum']}")

    return response,input_token_response,output_token_response


### 📈 Visualize the Aggregated Metrics

Let's fetch and plot the CloudWatch metrics for our System Inference Profile.

**What you'll see:**
- Combined metrics for all tenants (aggregated data)
- No way to distinguish between different tenants
- Single dimension: `ModelId = <system-inference-profile-id>`

**The Multi-Tenant Problem Visualized:**

Imagine if these metrics represented:
- 60% Tenant A (B2B Tech - high-volume user)
- 40% Tenant B (B2C Retail - medium-volume user)

With System Inference Profiles, **you can't tell!** All usage is combined into one metric stream.

In [ ]:
# Check CloudWatch metrics After  campaign generation
import time
print("⏳ Waiting 60 seconds for CloudWatch metrics to propagate...")
time.sleep(60)
print("✅ Wait complete - proceeding with metrics monitoring")

# Create graphs

response,input_token_response,output_token_response = fetch_metrices(Region, ModelId, 60, 60)

fig, axes = plt.subplots(3, 1, figsize=(12, 10))

# Plot Invocations
inv_data = sorted(response['Datapoints'], key=lambda x: x['Timestamp'])
inv_times = [dp['Timestamp'] for dp in inv_data]
inv_values = [dp['Sum'] for dp in inv_data]
axes[0].plot(inv_times, inv_values, marker='o', linewidth=2)  # Added linewidth
axes[0].set_title('Invocations over 1 hour')
axes[0].set_ylabel('Count')
axes[0].grid(True)

# Plot Input Tokens
input_data = sorted(input_token_response['Datapoints'], key=lambda x: x['Timestamp'])
input_times = [dp['Timestamp'] for dp in input_data]
input_values = [dp['Sum'] for dp in input_data]
axes[1].plot(input_times, input_values, marker='o', color='green', linewidth=2)  # Added linewidth
axes[1].set_title('Input Token Count over 1 hour')
axes[1].set_ylabel('Tokens')
axes[1].grid(True)

# Plot Output Tokens
output_data = sorted(output_token_response['Datapoints'], key=lambda x: x['Timestamp'])
output_times = [dp['Timestamp'] for dp in output_data]
output_values = [dp['Sum'] for dp in output_data]
axes[2].plot(output_times, output_values, marker='o', color='red', linewidth=2)  # Added linewidth
axes[2].set_title('Output Token Count over 1 hour')
axes[2].set_ylabel('Tokens')
axes[2].grid(True)

plt.tight_layout()
plt.show()


## Lab Summary: The Multi-Tenant Metrics Problem

### 🎯 What You Experienced

You just experienced the **fundamental limitation of System Inference Profiles** for multi-tenant applications:

**❌ The Problem:**
1. **All tenants share the same ModelId dimension** in CloudWatch
2. **No way to separate metrics** by tenant
3. **No way to track costs** per tenant
4. **No way to implement usage-based billing** accurately
5. **No way to monitor SLAs** per tenant

### 📊 What You Saw in CloudWatch

```
System Inference Profile Metrics:
├─ ModelId: us.anthropic.claude-sonnet-4-20250514-v1:0
│   ├─ Invocations: >1 (but which tenant?)
│   ├─ InputTokenCount: x (Tenant A? B?)
│   └─ OutputTokenCount: y (impossible to allocate!)
```

**The Reality:** These metrics represent **all tenants combined**. You cannot:
- Filter by specific tenant
- Calculate per-tenant costs
- Identify high-usage tenants
- Implement fair billing

### 💼 Business Impact

As **Alex, the Platform Engineer**, you need to tell your CEO:

1. **💰 Cost Allocation:** "We can't determine individual tenant costs"
2. **📊 Usage Tracking:** "We don't know which tenants are heavy users"
3. **🎯 Pricing Model:** "We can't implement accurate usage-based pricing"
4. **📉 SLA Monitoring:** "We can't track performance per tenant"

**This is a showstopper for any multi-tenant SaaS business!**

### 🚀 The Solution Preview

In **Lab 02**, you'll discover **Application Inference Profiles (AIP)** - the solution that enables:

✅ **Per-tenant model access** with unique identifiers 

✅ **Isolated CloudWatch metrics** for each tenant

✅ **Accurate cost tracking** per tenant

✅ **Usage-based billing** with confidence

✅ **Per-tenant SLA monitoring**

**The key difference:**
```python
# System Inference Profile (Lab 01 - Problem)
modelId = "us.anthropic.claude-sonnet-4-20250514-v1:0"  # Shared by all tenants

# Application Inference Profile (Lab 02 - Solution)  
tenant_a_modelId = "arn:aws:bedrock:region:account:application-inference-profile/tenant-a"  # Unique per tenant!
tenant_b_modelId = "arn:aws:bedrock:region:account:application-inference-profile/tenant-b"  # Unique per tenant!
```

### 💡 Key Takeaway

**System Inference Profiles are great for:**
- Single-tenant applications
- Internal tools
- Prototyping and development

**But for multi-tenant SaaS platforms, you need Application Inference Profiles to:**
- Track usage per tenant
- Allocate costs accurately
- Implement fair billing
- Monitor SLAs per tenant

**Ready to see the solution?** → [Continue to Lab 02: Application Inference Profiles Solution](lab-02-aip-solution-and-crud.ipynb)